ENTEROS

Cuando introduces un entero de Python en NumPy, este se convierte en un tipo nativo de NumPy llamado np.int32 (o np.int64, dependiendo del sistema operativo, la versión de Python y la magnitud de los inicializadores):

In [3]:
import numpy as np

arr=np.array([1, 2, 3]).dtype
print(arr)

int32


NumPy funciona mejor cuando el ancho de los elementos del array es fijo. Es más rápido y consume menos memoria, pero a diferencia de un entero ordinario de Python (que trabaja con aritmética de precisión arbitraria), los valores de un array se desbordarán cuando superen el valor máximo (o mínimo) para el tipo de dato correspondiente:

Los enteros con signo se desbordan silenciosamente en este momento, pero no hay garantía de que siempre lo hagan. El prefijo 'u' significa 'sin signo'

In [89]:
import numpy as np

arr = np.array([255], np.uint8) + 1 # 2**8-1 es el Entero màximo para uint8
print(f"arr= {arr} (dtype = uint8)") # dtype=uint8

arr1 = np.array([2**31-1]) #2**31-1 is el Entero máximo para int32
print(f"arr1= {arr1} (dtype = int32)") # dtype= int32

arr2 = np.array([2**31-1]) + 1 # o np.array([2**31-1], np.int32)+1 en linux
print(f"arr2= {arr2} (dtype = int32)") # dtype= int32

arr3 = np.array([2**63-1]) + 1 # siempre np.int64 since v > 2**32-1
print(f"arr3= {arr3} (dtype = int64)") # dtype= int64

arr= [0] (dtype = uint8)
arr1= [2147483647] (dtype = int32)
arr2= [-2147483648] (dtype = int32)
arr3= [-9223372036854775808] (dtype = int64)


Un tipo más amplio, y si no existe, lanza una advertencia de desbordamiento (para evitar inundar la salida con advertencias — solo una vez):

In [21]:
import numpy as np

arr=np.array([255], np.uint8) [0] + 1 # ok, promovido a int32(win)/int64(linux)
print(f"arr= {arr}") 

arr1=np.array([2**31-1]) [0] +1 # Cuidado!
print(f"arr1= {arr1}") 

arr2=np.array([2**63-1]) [0] + 1 # ok, advertido!
print(f"arr2= {arr2}") 

arr= 256
arr1= -2147483648
arr2= -9223372036854775808


C:\Users\REY\AppData\Local\Temp\ipykernel_17740\3644953772.py:6: RuntimeWarning: overflow encountered in scalar add
  arr1=np.array([2**31-1]) [0] +1 # Cuidado!
C:\Users\REY\AppData\Local\Temp\ipykernel_17740\3644953772.py:9: RuntimeWarning: overflow encountered in scalar add
  arr2=np.array([2**63-1]) [0] + 1 # ok, advertido!


Detección de los desbordamientos en 'escalares', conviertiendolo en error:

In [22]:
import numpy as np

with np.errstate(over='raise'):
    print(np.array([2**31-1]) [0] + 1)   

# Lanza un error con el mensaje: 
# FloatingPointError: Se encontró desbordamiento en la suma escalar     

FloatingPointError: overflow encountered in scalar add

Se puede suprimir temporalmente:

In [23]:
import numpy as np

with np.errstate(over='ignore'):
    print(np.array([2**31-1]) [0] + 1)   

-2147483648


O completamente:

In [31]:
import numpy as np
import warnings as warn

warn.filterwarnings('ignore','overflow')

NumPy incluye alias al estilo de C:
np.byte = np.int8
np.int16 = np.intc
np.int = np.int32 en Windows 64 bits pero es np.int64 en linux/MacOS de 64 bits
np.intp = np.int32 en Python de 32 bits pero = np.int64 en Python de 64 bits

Finalmente, si por alguna razón necesitas enteros de precisión arbitraria (enteros de Python) en ndarrays, NumPy también es capaz de hacerlo.

In [32]:
import numpy as np

a = np.array([10], dtype=object)
print(len(str(a**1000)))               # '[1000 ... 0]'

1003


Pero sin la mejora habitual en velocidad, ya que almacenará referencias en lugar de los números mismos, mantendrá la conversión de objetos Python al procesar, etc.

FLOATs

El tipo 'float' de Python es directamente compatible con 'np.float64' y el tipo 'complex' de Python — con 'np.complex128'.

Dos definiciones alternativas dan 15 y 17 dígitos para 'np.float64', 6 y 9 para 'np.float32', etc.

A día de hoy, 'np.float128' está disponible únicamente en Unix (no disponible en Windows).

In [33]:
import numpy as np

x = np.array([-1234.5])
y = 1/(1 + np.exp(-x))

print(y)

[0.]


Lo que esta advertencia intenta decirte es que NumPy es consciente de que matemáticamente hablando, 1/(1+exp(-x)) nunca puede ser cero, pero en este caso particular, debido a un desbordamiento, lo es.

Estas advertencias pueden ser "promovidas" a excepciones o silenciadas mediante 'errstate' o 'filterwarnings', como se describe en la sección de 'enteros' anteriormente —y quizás para este caso particular eso sería suficiente— pero si realmente deseas obtener el valor exacto puedes seleccionar un 'dtype' más amplio:

import numpy as np

x = np.array([-1234.5], dtype = float128)
y = 1/(1 + np.exp(-x))

qrray([7.30234068e-537], dtype = float128)

-- np.float128 está solo disponible en linux/MAcOS, no en Windows

Una de las cosas que distinguen a los Floats de los enteros es que son 'inexactos', no se pueden comparar dos floats con 'a==b' a menos que  estés seguro de que están representados exactamente.
Puedes esperar que los floats representen exactamente los enteros, pero solo hasta cierto nivel (limitado por el número de dígitos significativos):

El enfoque estándar para manejar este problema (así como la segunda fuente de inexactitud: el redondeo de los resultados de los cálculos) es compararlos con una tolerancia relativa (para comparar dos argumentos distintos de cero) y una tolerancia absoluta (si uno de los argumentos es cero). Para escalares, se maneja con 'math.isclose(a, b, *, rel_tol=1e-09, abs_tol=0.0)', y para arrays de NumPy existe una versión vectorizada 'np.isclose(a, b, rtol=1e-05, atol=1e-08)'. Es importante notar que las tolerancias tienen nombres y valores predeterminados diferentes.

Para datos financieros, el tipo 'decimal.Decimal' es útil ya que no involucra ninguna tolerancia adicional en absoluto.

In [40]:
import numpy as np
from decimal import Decimal as D

a = np.array([D('0.1'), D('0.2')]); 

print(a)
print(f"La suma de los decimales es: {a.sum()}")

[Decimal('0.1') Decimal('0.2')]
La suma de los decimales es: 0.3


Pero no es una solución universal: también tiene errores de redondeo (ver fuente #2 arriba). El único problema que resuelve es la representación exacta de números decimales a la que los humanos están acostumbrados.
Además, no soporta nada más complicado que operaciones aritméticas (aunque sí soporta logaritmos y raíces cuadradas) y es más lento que los floats. Para cálculos matemáticos puros se puede utilizar 'fractions.Fraction':

In [42]:
from fractions import Fraction

# Crear fracciones
frac1 = Fraction(1, 2)  # Representa 1/2
frac2 = Fraction(3, 4)  # Representa 3/4

print("Crear fracciones")
print("----------------")
print()
print("frac1 = Fraction(1, 2)  # Representa 1/2")
print("frac2 = Fraction(3, 4)  # Representa 3/4")
print()

# Suma de fracciones
suma = frac1 + frac2
print(f"Suma: {suma}")  # Output: Suma: 5/4

# Resta de fracciones
resta = frac2 - frac1
print(f"Resta: {resta}")  # Output: Resta: 1/4

# Multiplicación de fracciones
multiplicacion = frac1 * frac2
print(f"Multiplicación: {multiplicacion}")  # Output: Multiplicación: 3/8

# División de fracciones
division = frac1 / frac2
print(f"División: {division}")  # Output: División: 2/3

# Convertir fracción a punto flotante
flotante = float(frac1)
print(f"Convertir a punto flotante: {flotante}")  # Output: Convertir a punto flotante: 0.5

Crear fracciones
----------------

frac1 = Fraction(1, 2)  # Representa 1/2
frac2 = Fraction(3, 4)  # Representa 3/4

Suma: 5/4
Resta: 1/4
Multiplicación: 3/8
División: 2/3
Convertir a punto flotante: 0.5


Se pueden representar cualquier número 'racional' excepto 'PI' y 'e', para manipular estas constantes 'irracionales' debes usar: SymPy

Tanto Decimal como Fraction no son tipos nativos para NumPy, pero NumPy es capaz de trabajar con ellos con todas las comodidades, como multidimensiones e indexación avanzada, aunque a costa de una velocidad de procesamiento más lenta que la de los tipos nativos int S o float S. 

Los números complejos se tratan de la misma manera que los floats. Hay funciones adicionales con nombres intuitivos como 'np.real(z)', 'np.imag(z)', 'np.abs(z)', 'np.angle(z)' que funcionan tanto en escalares como en matrices completas.

La única diferencia con el complejo puro de Python, 'np.complex_', es que no funciona con enteros:

In [43]:
import numpy as np

x = np.array([1 + 2j])              # .dtype == np.complex128

print(x)

[1.+2.j]


Al igual que con los enteros, en arreglos de floats (y complejos) a veces es útil tratar ciertos valores como "faltantes". Los floats son más adecuados para almacenar datos anómalos: tienen un valor 'math.nan' (o 'np.nan' o 'float('nan')') que se puede almacenar junto con los valores numéricos "válidos".

La mayoría de las funciones estadísticas comunes tienen una versión resistente a `nan` (como `np.nansum`, `np.nanstd`, etc.), pero otras operaciones en esa columna o arreglo requerirían prefiltrado. 

También los nombres 'float96' / 'float128' son algo engañosos. En realidad, bajo la capucha no es '__float128', sino lo que sea que 'longdouble' signifique en la implementación local de C++. 

Para una mejor portabilidad, se recomienda utilizar el alias 'np.longdouble' en lugar de 'np.float96` / 'np.float128', porque internamente es lo que se utilizará de todos modos.

BOOLs

Los valores booleanos se almacenan como bytes individuales para mejorar el rendimiento. 'np.bool_' es un tipo separado del 'bool' de Python porque no necesita recuento de referencias ni un enlace a la clase base requerido para cualquier tipo puramente de Python. Entonces, si consideras que usar 8 bits para almacenar un bit de información es excesivo, mira esto:
En escenarios del mundo real, la tasa es menor: al empaquetar booleanos de NumPy en un arreglo, cada uno ocupará 1 byte, pero si empaquetas booleanos de Python en una lista, referenciará los mismos dos valores cada vez, costando efectivamente 8 bytes por elemento en x86_64.

STRINGs

Cuando inicializas un arreglo de NumPy con una lista de cadenas de Python, estas se almacenan en un dtype nativo fijo llamado 'np.str_'. Reservar un espacio necesario para que quepa la cadena más larga en cada elemento puede parecer derrochador (especialmente con la codificación fija UCS-4 en comparación con la elección 'dinámica' del ancho UTF en las cadenas de Python 'str').

In [46]:
import numpy as np

x = np.array(['abcde','x','y','x'], dtype = '<U5')      # 4 bytes por un caracter
                                                        # => 5*4 bytes por elemento

print(x)  

['abcde' 'x' 'y' 'x']


La abreviatura '<U4' proviene del llamado protocolo de arreglo introducido en 2005. Significa ‘Little-endian  USC-4 encoded string, de longitud 5 elementos' (USC-4≈UTF-32, una codificación de ancho fijo de 4 bytes por carácter). Cada tipo de NumPy tiene una abreviatura —tan difícil de leer como esta—, pero afortunadamente han adoptado nombres legibles por humanos al menos para los dtypes más utilizados.

Otra opción es mantener referencias a cadenas de Python ('str S') en un arreglo de objetos de NumPy:

In [90]:
import numpy as np

np.array(['abcde','x','y','x'], object)  # 4 bytes por caracter ascii
                                         # => 49+len(el) por elemento

array(['abcde', 'x', 'y', 'x'], dtype=object)

Si estás trabajando con una secuencia bruta de bytes, NumPy tiene una versión de longitud fija del tipo bytes de Python llamada np.bytes_:

In [91]:
import numpy as np

np.array([b'abcde',b'x',b'y',b'x'])     # 1 byte por caracter ascii
                                            # => 5 byte por elemento
                                            

array([b'abcde', b'x', b'y', b'x'], dtype='|S5')

Aquí, |S5 significa 'secuencia de bytes de longitud 5 sin aplicabilidad de endianness'.

En cuanto a los tipos nativos 'np.str_' y 'np.bytes_', NumPy tiene varias operaciones comunes de cadenas. Estas reflejan los métodos de cadena de Python, residen en el módulo 'np.char' y operan sobre todo el array:

In [92]:
import numpy as np

np.char.upper(np.array([['a','b'],['c','d']]))


array([['A', 'B'],
       ['C', 'D']], dtype='<U1')

En el caso de las cadenas en modo objeto, los bucles deben realizarse en el nivel de Python:

In [94]:
import numpy as np

a = np.array([['a','b'],['c','d']], object)
np.vectorize(lambda x: x.upper(), otypes = [object]) (a)


array([['A', 'B'],
       ['C', 'D']], dtype=object)

Primera Sentencia:
a = np.array([['a', 'b'], ['c', 'd']], object)

1. 'np.array': Esta función de NumPy crea un array a partir de la lista proporcionada.
2. '[['a', 'b'], ['c', 'd']]': Es una lista de listas que representa una matriz de 2x2 con elementos de tipo cadena.
3. 'object': Este argumento especifica que los elementos del array deben ser tratados como objetos de Python en lugar de utilizar un tipo de dato NumPy específico como 'np.int32' o 'np.float64'.

Como resultado, 'a' será una matriz 2x2 donde cada elemento es un objeto de cadena:

array([['a', 'b'],
       ['c', 'd']], dtype=object)

Segunda Sentencia:
b = np.vectorize(lambda x: x.upper(), otypes=[object])(a)

1. 'np.vectorize': Esta es una función de NumPy que toma una función de Python y devuelve una versión "vectorizada" de esa función. Esto significa que la función se puede aplicar a arrays de NumPy de manera elemento por elemento, como si estuviera trabajando con un bucle for en Python, pero de una manera que es más eficiente y se siente más natural en NumPy.
2. 'lambda x: x.upper()': Esta es una función lambda que toma un argumento 'x' y devuelve 'x.upper()', que es el método para convertir cadenas a mayúsculas.
3. 'otypes=[object]': Este argumento le dice a 'np.vectorize' que el tipo de salida de la función será un objeto. Esto es necesario porque estamos trabajando con cadenas, que son objetos en Python.
4. '(a)': Esto aplica la función vectorizada a cada elemento del array 'a'.

Como resultado, 'b' será un array donde cada elemento de 'a' ha sido convertido a mayúsculas:

array([['A', 'B'],
       ['C', 'D']], dtype=object)

Explicación Paso a Paso:
- Definición del array 'a': Se crea un array de NumPy con elementos de tipo cadena.
- Vectorización de la función 'lambda x: x.upper()': Se utiliza 'np.vectorize' para permitir que esta función lambda sea aplicada a cada elemento del array de manera eficiente.
- Aplicación de la función vectorizada a 'a': La función se aplica a cada elemento del array 'a', transformando cada letra minúscula en mayúscula y almacenando el resultado en 'b'.

Este proceso convierte todos los caracteres del array original 'a' a mayúsculas y almacena el resultado en 'b'.

Según pruebas de rendimiento, las operaciones básicas funcionan algo más rápido con 'str' que con 'np.str'.

Una marca de tiempo, también conocida como tiempo Unix, representa el número de segundos transcurridos desde el 1 de enero de 1970. Puede contar el tiempo con una granularidad configurable, que va desde años hasta attosegundos, y siempre está representada por un único número int64.

•	La granularidad de años significa "simplemente contar los años", sin ninguna mejora real frente a almacenar los años como un entero.
•	La granularidad de días es equivalente a 'datetime.date' en Python.
•	Microsegundos son equivalentes a 'datetime.datetime' en Python.
Y todo lo que está por debajo es exclusivo de `np.datetime64`.
Al crear una instancia de 'np.datetime64', NumPy elige la granularidad más gruesa que aún pueda contener esos datos:

DATETIMES

In [64]:
import numpy as np
from datetime import datetime as dt

t = np.datetime64('today')  
print("today")        
print(f"Granularidad de dias (en hora local UTC+7): {t}")
print()
print("now") 
t1 = np.datetime64('now')            
print(f"Granularidad de segundos (en UTC): {t1}")
print()
print("dt.utcnow()") 
t2 = np.datetime64(dt.utcnow())  # Me indica que esta función está en desuso y será eliminada en una versión futura de python
print(f"Granularidad de microsegundo: {t2}")
print()
print("2021-12-24 18:14:23.404438") 
t3 = np.datetime64('2021-12-24 18:14:23.404438')  
print(f"Granularidad de nanosegundos: {t3}")



today
Granularidad de dias (en hora local UTC+7): 2024-06-22

now
Granularidad de segundos (en UTC): 2024-06-22T20:19:04

dt.utcnow()
Granularidad de microsegundo: 2024-06-22T20:19:04.589531

2021-12-24 18:14:23.404438
Granularidad de nanosegundos: 2021-12-24T18:14:23.404438


C:\Users\REY\AppData\Local\Temp\ipykernel_17740\1101429202.py:13: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  t2 = np.datetime64(dt.utcnow())  # Me indica que esta función está en desuso y será eliminada en una versión futura de python


Este formato exacto o variaciones mínimas del mismo (ver los 'principios generales' de la página de Wikipedia de ISO 8601).
Al crear un array, decides si estás de acuerdo con la granularidad que NumPy ha elegido para ti o insistes, por ejemplo, en nanosegundos o algo así, y te proporcionará 2⁶³ momentos equidistantes medidos en las unidades de tiempo correspondientes a cada lado del 1 de enero de 1970.

Para obtener un campo particular de un escalar datetime64/timedelta64, puedes convertirlo a un datetime convencional:

In [72]:
f = np.datetime64('2021-12-24 18:14:23').item()
m = f.month
print(f"Fecha = {f}")
print(f"Mes = {m}")


Fecha = 2021-12-24 18:14:23
Mes = 12


Para los arrays como este uno:

In [74]:
import numpy as np
a = np.arange(np.datetime64('2021-01-20'),
           np.datetime64('2021-12-20'),
           np.timedelta64(90, 'D')); a

print(f"{a},  dtype = 'datetime64[D]'")

['2021-01-20' '2021-04-20' '2021-07-19' '2021-10-17'],  dtype = 'datetime64[D]'


Puedes hacer conversiones entre subtipos de np.datetime64 (más rápido) o usar Pandas (de 2 a 4 veces más lento):

Aquí tienes una función útil que descompone un arreglo de datetime64 en un arreglo de 7 columnas enteras (años, meses, días, horas, minutos, segundos, microsegundos):

In [76]:
import numpy as np
from datetime import datetime as dt

def dt2cal(dt):
    # Asignar salida
    out = np.empty(dt.shape + (7,), dtype = "u4")
    # Descomponer calendario
    Y, M, D, h, m, s = [dt.astype(f"M8[{x}]") for x in "YMDhms"]
    out[..., 0] = Y + 1970   # Año Gregoriano
    out[..., 1] = (M-Y) + 1  # fecha
    out[..., 2] = (D-M) + 1  # Mes
    out[..., 3] = (dt-D).astype("m8[h]")  # hora
    out[..., 4] = (dt-h).astype("m8[m]")  # minuto
    out[..., 5] = (dt-m).astype("m8[s]")  # segundo
    out[..., 6] = (dt-s).astype("m8[us]")  # microsegundo
    
    return out

a = np.arange(np.datetime64('2021-01-20'),
           np.datetime64('2021-12-20'),
           np.timedelta64(90, 'D')); a

dt2cal(a) 
    
    
    

array([[2021,    1,   20,    0,    0,    0,    0],
       [2021,    4,   20,    0,    0,    0,    0],
       [2021,    7,   19,    0,    0,    0,    0],
       [2021,   10,   17,    0,    0,    0,    0]], dtype=uint32)

Un par de detalles a tener en cuenta con las fechas:

1.	Aunque se admiten años bisiestos,

In [77]:
import numpy as np
from datetime import datetime as dt

np.array(['2020-03-01','2022-03-01', '2024-03-01'], np.datetime64)- \
np.array(['2020-02-01','2022-02-01', '2024-02-01'], np.datetime64)



array([29, 28, 29], dtype='timedelta64[D]')

El módulo time los soporta solo de manera formal (acepta el segundo 60, pero da intervalos incorrectos). Parece que hasta ahora solo astropy los procesa correctamente.

Parece que hasta ahora solo astropy los procesa correctamente.

In [80]:
from astropy.time import Time

(Time('2017-01-01')-Time('2016-12-31 23:59')).sec

61.00000000001593

Otros se adhieren al calendario gregoriano proleptico con sus exactamente 86400 segundos SI al día, que ya ha ganado aproximadamente medio minuto de diferencia con el tiempo universal desde 1970 debido a las irregularidades en la rotación de la Tierra.
Las implicaciones prácticas de usar este calendario son:
•	Error al calcular intervalos que incluyen uno o más segundos intercalares.
•	Excepción al intentar construir un datetime64 a partir de una marca de tiempo tomada durante un segundo intercalar.

2.	Dado que tanto np.datetime64 como np.timedelta64 tienen el mismo ancho, se debe tener cuidado con los timedelta grandes:

In [82]:
import numpy as np

np.datetime64('2262-01-01', 'ns') - np.datetime64('1678-01-01', 'ns')

numpy.timedelta64(-17537673709551616,'ns')

Finalmente, ten en cuenta que todos los tiempos en np.datetime64 son 'ingenuos' (naive): no son 'conscientes' del horario de verano (por lo que se recomienda almacenar todos los datetimes en UTC) y no pueden convertirse de una zona horaria a otra (usa pytz para conversiones de zona horaria):

In [99]:
import numpy as np
from datetime import datetime

a = np.arange(np.datetime64('2021-01-20'),
           np.datetime64('2021-12-20'),
           np.timedelta64(90, 'D')); a

np.datetime_as_string(a)

array(['2021-01-20', '2021-04-20', '2021-07-19', '2021-10-17'],
      dtype='<U28')

COMBINATIONS THEREOF

Un "structured array" en NumPy es un arreglo con un dtype personalizado hecho a partir de los tipos descritos anteriormente como bloques básicos de construcción (similar a struct en C). Un ejemplo típico es el color de píxel RGB: un tipo de longitud 3 bytes (generalmente 4 por alineación), en el cual los colores pueden ser accesibles por nombre:

In [117]:
import numpy as np

rgb = np.dtype([('x',np.uint8),('y', np.uint8), ('z',np.uint8)])
a = np.zeros(5, rgb); a

array([(0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0)],
      dtype=[('x', 'u1'), ('y', 'u1'), ('z', 'u1')])

In [111]:
a[0]

(0, 0, 0)

In [119]:
a[0]['x']

0

In [122]:
a[0]['x'] = 10
a

array([(10, 0, 0), ( 0, 0, 0), ( 0, 0, 0), ( 0, 0, 0), ( 0, 0, 0)],
      dtype=[('x', 'u1'), ('y', 'u1'), ('z', 'u1')])

In [123]:
a['z'] = 5
a

array([(10, 0, 5), ( 0, 0, 5), ( 0, 0, 5), ( 0, 0, 5), ( 0, 0, 5)],
      dtype=[('x', 'u1'), ('y', 'u1'), ('z', 'u1')])

Los tipos para los arrays estructurados no necesariamente tienen que ser homogéneos e incluso pueden incluir subarrays.
Con los arrays estructurados y los recarrays se puede obtener la 'apariencia' básica de un DataFrame de Pandas:
•	Puedes acceder a las columnas por nombres,
•	Realizar cálculos aritméticos y estadísticos con ellas,
•	Manejar eficientemente valores faltantes,
•	Algunas operaciones son más rápidas en NumPy que en Pandas.
Pero carecen de:
•	agrupación (excepto lo que ofrece itertools.groupby),
•	el poderoso índice y MultiIndex de Pandas (por lo que no hay tablas dinámicas) y
•	otras comodidades como ordenar convenientemente, etc.

recarray
En NumPy, un 'recarray' (o "record array") es una subclase de arrays estructurados que permite el 
acceso a los campos de la estructura utilizando atributos en lugar de índices. Esto puede hacer que 
el acceso a los datos sea más conveniente y legible.

Ejemplo:

In [128]:
import numpy as np

# Definir un dtype estructurado
dtype = np.dtype([('x', np.int32), ('y', np.float64), ('name', 'U10')])

# Crear un recarray directamente
recarray = np.recarray((3,), dtype=dtype)

# Asignar valores
recarray.x = [1, 2, 3]
recarray.y = [2.5, 3.6, 7.8]
recarray.name = ['Alice', 'Bob', 'Cathy']

# Acceder a los campos como atributos
print(recarray.x)  # Salida: [1 2 3]
print(recarray.y)  # Salida: [2.5 3.6 7.8]
print(recarray.name)  # Salida: ['Alice' 'Bob' 'Cathy']


[1 2 3]
[2.5 3.6 7.8]
['Alice' 'Bob' 'Cathy']


TYPE CHECKS

Una forma de verificar el tipo de arreglo de NumPy es utilizar isinstance con uno de sus elementos:

In [130]:
import numpy as np

a = np.array([1, 2, 3])
v = a[0]
isinstance(v, np.int32)

True

Todos los tipos de NumPy están interconectados en un árbol de herencia que se muestra en la parte superior del artículo (azul=clases abstractas, verde=tipos numéricos, amarillo=otros), por lo tanto, en lugar de especificar una lista completa de tipos como isinstance(v, [np.int32, np.int64, etc.]), puedes escribir controles de tipo más compactos como

In [131]:
isinstance(v, np.floating)

False

In [132]:
isinstance(v, np.complexfloating)

False

El inconveniente de este método es que solo funciona contra un valor del arreglo, no contra el arreglo en sí. Esto no es útil cuando el arreglo está vacío, por ejemplo. Verificar el tipo del arreglo es más complicado.

Para tipos básicos, el operador == hace el trabajo para una única verificación de tipo:


In [133]:
a.dtype == np.int32

True

In [134]:
a.dtype == np.int64

False

Y el operador 'in'  para verificar contra un grupo de tipos:

In [139]:
import numpy as np

# Definir xdtype, por ejemplo, np.double
xdtype = np.double

# Verificar si xdtype es uno de los tipos de punto flotante de NumPy
if xdtype in (np.half, np.single, np.double, np.longdouble):
    print("xdtype es un tipo de punto flotante de NumPy.")
else:
    print("xdtype no es un tipo de punto flotante de NumPy.")


xdtype es un tipo de punto flotante de NumPy.


Pero para tipos más sofisticados como np.str_ o np.datetime64, no lo hacen.
La forma recomendada⁴ de verificar el dtype contra los tipos abstractos es

In [140]:
np.issubdtype(a.dtype, np.integer)


True

In [141]:
np.issubdtype(a.dtype, np.floating)

False

Funciona con todos los tipos nativos de NumPy, pero la necesidad de este método parece algo no obvia: ¿qué tiene de malo el buen y viejo isinstance? Obviamente, la complejidad de la estructura de herencia de los dtypes (¡se construyen 'sobre la marcha'!) no les permitió hacerlo según el principio de la menor sorpresa.

Otro método es utilizar el diccionario np.typecodes (no documentado, pero utilizado en las bases de código de SciPy/NumPy, por ejemplo aquí). El árbol que representa tiene muchas menos ramificaciones:

In [142]:
np.typecodes

{'Character': 'c',
 'Integer': 'bhilqp',
 'UnsignedInteger': 'BHILQP',
 'Float': 'efdg',
 'Complex': 'FDG',
 'AllInteger': 'bBhHiIlLqQpP',
 'AllFloat': 'efdgFDG',
 'Datetime': 'Mm',
 'All': '?bhilqpBHILQPefdgFDGSUVOMm'}

Su aplicación principal es generar arreglos con dtypes específicos para propósitos de prueba, pero también se puede utilizar para distinguir entre diferentes grupos de dtypes:

In [143]:
a.dtype.char in np.typecodes['AllInteger']

True

In [144]:
a.dtype.char in np.typecodes['Datetime']

False

Nota que usar 'a.dtype.kind` en lugar de 'a.dtype.char' es un error: np.zeros(1, dtype=np.uint8).dtype.kind == 'u' está ausente en 'np.typecodes', while <...>.char == 'B' está listado allí.
Uno de los inconvenientes de este método es que los tipos bool, strings, bytes, objects y voids ('?', 'U', 'S', 'O' y 'V', respectivamente) no tienen claves dedicadas en el diccionario.
Este enfoque parece más de hackeo pero menos mágico que 'issubdtype'.